In [6]:
import pandas as pd
df=pd.read_csv('df_en_to_fr_lemma.csv')
df.type.replace(list(df.type.unique()),[i for i in range(21)],inplace=True)
df.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,filename,type,text_image,type_char,text_image_fr,text_image_fr_new,text_image_fr_new_lemma
0,0,0,0,0,img_0000000.jpg,0,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,facture,[{'translation_text': 'facture logo joanna bin...,facture logo joanna binet 48 coubertin 31400 p...,factur logo joanna binet 48 coubertin 31400 pa...
1,1,1,1,1,img_0000001.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture,[{'translation_text': 'joanna binet 48 coubert...,joanna binet 48 coubertin 31400 paris facture ...,joanner binet 48 coubertin 31400 pari factur f...
2,2,2,2,2,img_0000002.jpg,0,FACTURE\n\nMon entreprise : Nom de la société\...,facture,[{'translation_text': 'facture mon entreprise ...,facture mon entreprise : nom de la société adr...,factur mon entreprise : nom de le société adre...
3,3,3,3,3,img_0000003.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture,[{'translation_text': 'joanna binet 48 coubert...,joanna binet 48 coubertin 31400 paris facture ...,joanner binet 48 coubertin 31400 pari factur à...
4,4,4,4,4,img_0000004.jpg,0,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,facture,[{'translation_text': 'payer en ligne > factur...,payer en ligne > facture no sfideli f/05 2020/...,payer en ligne > factur no sfideli f/05 2020/0...


In [8]:
#nettoyage de text 3
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
stop_words.update(["." , ",",':'])
stop_words=list(stop_words)
stop_words.extend(stopwords.words('french'))


def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(str(w).lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()
df['text_image_fr_new_lemma'] = df.text_image_fr_new_lemma.apply(lambda x :preprocess_sentence(x))
df.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,filename,type,text_image,type_char,text_image_fr,text_image_fr_new,text_image_fr_new_lemma
0,0,0,0,0,img_0000000.jpg,0,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,facture,[{'translation_text': 'facture logo joanna bin...,facture logo joanna binet 48 coubertin 31400 p...,factur logo joanna binet coubertin pari factur...
1,1,1,1,1,img_0000001.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture,[{'translation_text': 'joanna binet 48 coubert...,joanna binet 48 coubertin 31400 paris facture ...,joanner binet coubertin pari factur facturer c...
2,2,2,2,2,img_0000002.jpg,0,FACTURE\n\nMon entreprise : Nom de la société\...,facture,[{'translation_text': 'facture mon entreprise ...,facture mon entreprise : nom de la société adr...,factur entreprise nom societe adress postal ad...
3,3,3,3,3,img_0000003.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture,[{'translation_text': 'joanna binet 48 coubert...,joanna binet 48 coubertin 31400 paris facture ...,joanner binet coubertin pari factur cendrillon...
4,4,4,4,4,img_0000004.jpg,0,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,facture,[{'translation_text': 'payer en ligne > factur...,payer en ligne > facture no sfideli f/05 2020/...,payer ligne factur sfideli date creation date ...


In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(df.text_image_fr_new_lemma,df.type,test_size=0.2,random_state=222)

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
vect=CountVectorizer()
tfidf=TfidfTransformer()
X_train=vect.fit_transform(X_train)
X_test=vect.transform(X_test)
X_train=tfidf.fit_transform(X_train)
X_test=tfidf.transform(X_test)

In [37]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

#param={ 'C':np.arange(1,200,1)}
clf=LogisticRegression(C=187)
#grid_clf=GridSearchCV(clf,param,cv=3)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        11
           1       0.96      0.87      0.91        30
           2       1.00      0.75      0.86         4
           3       0.61      0.85      0.71        27
           4       0.62      0.89      0.73        44
           5       0.79      0.54      0.64        35
           6       0.67      0.92      0.78        66
           7       0.67      0.50      0.57         4
           8       0.50      1.00      0.67         2
           9       1.00      0.38      0.55         8
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00         1
          12       0.75      0.75      0.75         4
          13       0.00      0.00      0.00         6
          14       0.40      0.22      0.29         9
          15       0.00      0.00      0.00         8
          16       0.50      0.20      0.29        10
          17       0.14    

C:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `z

In [38]:
#  model SVC
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

param={'kernel':('linear', 'rbf')}
clf=SVC()
grid_clf=GridSearchCV(clf,param,cv=3)
grid_clf.fit(X_train, y_train)
y_pred=grid_clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96        11
           1       1.00      0.80      0.89        30
           2       1.00      0.50      0.67         4
           3       0.53      0.67      0.59        27
           4       0.95      0.80      0.86        44
           5       0.48      0.89      0.63        35
           6       0.56      0.95      0.70        66
           7       1.00      0.25      0.40         4
           8       0.67      1.00      0.80         2
           9       0.00      0.00      0.00         8
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00         1
          12       1.00      0.75      0.86         4
          13       0.00      0.00      0.00         6
          14       1.00      0.11      0.20         9
          15       0.00      0.00      0.00         8
          16       1.00      0.20      0.33        10
          17       0.00    

C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
# model GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()

X_train, y_train=ros.fit_resample(X_train, y_train)
clf=GradientBoostingClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.89      0.83      0.86        30
           2       1.00      0.75      0.86         4
           3       0.95      0.67      0.78        27
           4       0.91      0.68      0.78        44
           5       0.36      0.89      0.52        35
           6       0.85      0.80      0.83        66
           7       0.40      0.50      0.44         4
           8       0.00      0.00      0.00         2
           9       0.60      0.38      0.46         8
          10       1.00      0.43      0.60         7
          11       0.00      0.00      0.00         1
          12       0.33      0.25      0.29         4
          13       1.00      0.50      0.67         6
          14       0.20      0.22      0.21         9
          15       0.38      0.38      0.38         8
          16       0.75      0.30      0.43        10
          17       0.00    

C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
"""partie Embedding"""

In [30]:
import numpy as np
from keras.utils import to_categorical

X_train=X_train.todense()
X_test=X_test.todense()

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)


In [31]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
inputs = Input(shape = (X_train.shape[1]), name = "Input")
dense1=Dense(units =1000, activation = "tanh", name = "Couche_1")
dense2=Dense(units = 500, activation = "tanh", name = "Couche_2")
dense3=Dense(units = 250, activation = "tanh", name = "Couche_3")
dense4=Dense(units = 100, activation = "tanh", name = "Couche_4")
dense5=Dense(units = 21, activation = "softmax", name = "Couche_5")
x = dense1(inputs)
x = dense2(x)
x = dense3(x)
x = dense4(x)
outputs = dense5(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 25279)]           0         
                                                                 
 Couche_1 (Dense)            (None, 1000)              25280000  
                                                                 
 Couche_2 (Dense)            (None, 500)               500500    
                                                                 
 Couche_3 (Dense)            (None, 250)               125250    
                                                                 
 Couche_4 (Dense)            (None, 100)               25100     
                                                                 
 Couche_5 (Dense)            (None, 21)                2121      
                                                                 
Total params: 25,932,971
Trainable params: 25,932,971
Non-tra

In [32]:
model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])

In [33]:
model_fit=model.fit(X_train,y_train,epochs = 20, batch_size = 56)

Epoch 1/20
22/22 [==============================] - 3s 92ms/step - loss: 2.0639 - accuracy: 0.4709
Epoch 2/20
22/22 [==============================] - 2s 96ms/step - loss: 0.6998 - accuracy: 0.8181
Epoch 3/20
22/22 [==============================] - 2s 98ms/step - loss: 0.2634 - accuracy: 0.9527
Epoch 4/20
22/22 [==============================] - 2s 92ms/step - loss: 0.1329 - accuracy: 0.9684
Epoch 5/20
22/22 [==============================] - 2s 95ms/step - loss: 0.1062 - accuracy: 0.9684
Epoch 6/20
22/22 [==============================] - 2s 93ms/step - loss: 0.1015 - accuracy: 0.9718
Epoch 7/20
22/22 [==============================] - 2s 100ms/step - loss: 0.0986 - accuracy: 0.9734
Epoch 8/20
22/22 [==============================] - 2s 93ms/step - loss: 0.0860 - accuracy: 0.9701
Epoch 9/20
22/22 [==============================] - 2s 99ms/step - loss: 0.0880 - accuracy: 0.9718
Epoch 10/20
22/22 [==============================] - 2s 98ms/step - loss: 0.0820 - accuracy: 0.9709
Epoch 11

In [34]:
from sklearn.metrics import classification_report

Y_pred=model.predict(X_test)
y_pred_class=Y_pred.argmax(axis=1)
y_test_class=y_test.argmax(axis=1)
print(classification_report(y_test_class, y_pred_class))

10/10 [==============================] - 0s 17ms/step
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        16
           1       0.78      0.90      0.84        20
           2       0.56      0.56      0.56         9
           3       0.57      0.72      0.63        18
           4       0.69      0.94      0.80        48
           5       0.86      0.45      0.59        40
           6       0.68      0.95      0.79        63
           7       0.50      0.60      0.55         5
           8       0.80      0.36      0.50        11
           9       0.50      0.17      0.25         6
          10       0.00      0.00      0.00         7
          11       0.89      0.89      0.89         9
          12       1.00      0.50      0.67         4
          13       0.00      0.00      0.00         5
          14       0.00      0.00      0.00         5
          15       0.57      0.67      0.62         6
          16       0.60    